In [1]:
import ee
# ใน Google Colab ต้องใช้ ee.Authenticate() แทน earthengine authenticate เพราะ Colab ไม่มี CLI เหมือนเครื่อง Local
ee.Authenticate()

# ตรวจสอบและเริ่มต้นการเชื่อมต่อ GEE
try:
    ee.Initialize(project="ee-sakda-451407")
except Exception as e:
    ee.Authenticate() # หากยังไม่ได้ยืนยันตัวตน ให้ทำการยืนยัน
    ee.Initialize(project="ee-sakda-451407")  

In [3]:
# Define region and date range
thailand = ee.FeatureCollection("FAO/GAUL/2015/level1") \
    .filter(ee.Filter.eq('ADM0_NAME', 'Thailand'))
north = thailand.filter(ee.Filter.inList('ADM1_NAME', ['Nan']))

start_date = '2016-01-01'
end_date = '2022-12-31'

# Monthly precipitation aggregation
precip = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
    .filterDate(start_date, end_date) \
    .map(lambda img: img.set('month', img.date().format('YYYY-MM')))

monthlyPrecip = precip.map(lambda img:
    precip.filter(ee.Filter.calendarRange(
        img.date().get('month'), img.date().get('month'), 'month')
    ).sum() \
    .set('system:time_start', img.date().millis()) \
    .set('month', img.date().format('YYYY-MM'))
)

# Evapotranspiration data
et = ee.ImageCollection("MODIS/061/MOD16A2GF") \
    .filterDate(start_date, end_date) \
    .map(lambda img: img.select('ET').multiply(0.1) \
        .set('month', img.date().format('YYYY-MM')))

# NDVI data
ndvi = ee.ImageCollection("MODIS/061/MOD13Q1") \
    .filterDate(start_date, end_date) \
    .map(lambda img: img.select('NDVI').multiply(0.0001) \
        .set('month', img.date().format('YYYY-MM')))

# Runoff data aggregation
runoff = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H") \
    .filterDate(start_date, end_date) \
    .select("Qs_acc") \
    .map(lambda img: img.set('month', img.date().format('YYYY-MM')))

monthlyRunoff = runoff.map(lambda img:
    runoff.filter(ee.Filter.calendarRange(
        img.date().get('month'), img.date().get('month'), 'month')
    ).sum() \
    .set('system:time_start', img.date().millis()) \
    .set('month', img.date().format('YYYY-MM'))
)

# Generate sampling points
grid = ee.FeatureCollection.randomPoints(north, 500)

# Combine datasets by month
joined = monthlyPrecip.map(lambda img:
    img.addBands(
        et.filter(ee.Filter.eq('month', img.get('month'))).first()
    ).addBands(
        ndvi.filter(ee.Filter.eq('month', img.get('month'))).first()
    ).addBands(
        monthlyRunoff.filter(ee.Filter.eq('month', img.get('month'))).first()
    ).set('month', img.get('month'))
)

# Create training dataset
training = joined.map(lambda img:
    img.sampleRegions(
        collection=grid,
        scale=5000,
        geometries=True
    ).map(lambda feat: feat.set('month', img.get('month')))
).flatten()

# Export results
task = ee.batch.Export.table.toDrive(
    collection=training,
    description='CNN_LSTM_WaterBalance_Training',
    fileFormat='CSV',
    folder='_GEE',
)
task.start()